# HERE TEST USING THE 'RESULTS' OBJECT FROM SAMPLE DATA JSON

### NOTE: ignore the first object in json i.e 'final results.json' as its the city itself

### Imports 

In [1]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import pandas as pd
from dotenv import load_dotenv
import os
import json

### Load env file with api key

In [4]:
load_dotenv() # reading all vaiables from .env file (api key)

True

### Load llm and test it 

In [7]:
OPENAPI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [8]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAPI_API_KEY, temperature=0.2, max_tokens=2048)

In [9]:
llm.invoke("if active respond with active")

AIMessage(content='Active', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 12, 'total_tokens': 14, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'stop', 'logprobs': None}, id='run-7a8b2eae-36e8-4108-9d67-650c5a71c428-0', usage_metadata={'input_tokens': 12, 'output_tokens': 2, 'total_tokens': 14, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Load JSON and print it 

In [ ]:
# Open and read the JSON file
with open('./final_results.json', 'r') as file:
    results_data = json.load(file)
# Print the data
print(results_data)

### Get Test Objects and Properties From json 

In [20]:
place = results_data[1]  # first location 
location = place['geometry']['location']
print("place data")
print(place)
print("location data")
print(location)
lat, lon = location['lat'], location['lng']
print(f"Latitude: {lat}, Longitude: {lon}")

place data
{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 43.6547284, 'lng': -79.3824216}, 'viewport': {'northeast': {'lat': 43.6560367802915, 'lng': -79.38150846970849}, 'southwest': {'lat': 43.65333881970851, 'lng': -79.38420643029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet', 'name': 'Marriott Downtown at CF Toronto Eaton Centre', 'photos': [{'height': 320, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116195274436403701222">Marriott Downtown at CF Toronto Eaton Centre</a>'], 'photo_reference': 'AWYs27xwzQJBr09DZPNdeVIRek7sH-gocDDlsoUul70nzJlo4iHnI3H2tWlYpBW9ahVdrcxFo93oc119DxNqVJudW9rZlSwcKfdF0H3GCZzjDVmt4jn3sjUeGDoyGXQuA9urY7BB7gLT9b5hqCkf4ag70MyiTBvwdeDE9Hz8xjyHuSCuEiCD', 'width': 480}], 'place_id': 'ChIJ1WIA7Q7bKk0R9JK-tM30kE0', 'plus_code': {'compound_code': 'MJ39